## 1.2. Clip GeoTIFF Map
___
Last edited: 05/02/2022 by Jeongkyung Won<br>
This code does: 
 *  clips 1970 NGII GeoTIFF file into 2㎢ grids and directly save them in png format
 *  converts black pixels in each clipped maps to transparent background

*NGII: National Geographic Information Institute, KR

In [ ]:
pip install geopandas

In [ ]:
import geopandas as gpd
import fiona,os
from osgeo import gdal
from tqdm import notebook
from os import listdir

In [ ]:
import requests
import zipfile
from urllib.parse import unquote # for url string conversion
import re
from tqdm import tqdm

def download_dropbox(url, folder):
  ## version 2.0, last modified by Hyunjoo Yang (hyang@sogang.ac.kr) on Jan. 14 2022
  ## This function downloads dropbox shareable link to a local folder (tested for file downloading, but not for shared folder!)
  # url: dropbox shareable link for downloading
  # folder: where to download
  
  headers = {'user-agent': 'Wget/1.16 (linux-gnu)'}
  r = requests.get(url, stream=True, headers=headers)
  
  # convert dropbox shareable link ("dl=0" to "dl=1")
  url = url.replace("?dl=0", "?dl=1")

  # check if the url returns valid status code (200)
  if r.status_code == 200:
    print('The url is valid.')

    # grab filename from the url, using regular expressions (and replace space to "_" )
    file_name = unquote(re.search(r'\/([^\/]+\.([\w]+))\?dl=([01])$', url).group(1)).replace(" ", "_")

    folder_n_fname = os.path.join(folder, file_name)

    # download
    print('Begin downloading < {} >'.format(folder_n_fname))


    # Total size in bytes.
    total_size = int(r.headers.get('content-length', 0))
    block_size = 1024  # 1 Kibibyte
    t = tqdm(total=total_size, unit='B', unit_scale=True)
    with open(file_name, 'wb') as f:
      for data in r.iter_content(block_size):
        t.update(len(data))
        f.write(data)
    t.close()

    print('Finished downloading < {} >'.format(folder_n_fname))

    # unzip if zip file
    if file_name.endswith('.zip'):
      print('Extracting zip file...')
      with zipfile.ZipFile(file_name,"r") as zip_ref:
        zip_ref.extractall(folder)
    else:
      if not os.path.exists(folder):
        os.makedirs(folder)
      shutil.move(file_name, folder_n_fname)

    print('Done!')
  else:
    raise ValueError('Nothing to download: dropbox link is not valid. Check the URL link again')

In [ ]:
## dropbox shareable url link

url_tiff = 'https://www.dropbox.com/s/v806q31pv7g9vgh/NGII_1970_EPSG_4326.zip?dl=0' # NGII_1970_EPSG_4326.tif
url_grid='https://www.dropbox.com/s/rkosd9lbyy8biua/pop_grid_2km_5179.zip?dl=0' # NGII population 2㎢ grid

# file path for download and unzip
tiff_path = './tiff'
grid_path = './pop_grid'

# download dropbox shared file and unzip it
download_dropbox(url_tiff, tiff_path)
download_dropbox(url_grid, grid_path)

In [ ]:
pop_grid_dr = "./pop_grid/population_grid_combined_2km.shp"
mask_dr = "./cookie_mold/" 

CRS = 'EPSG:5179'

raster = "./tiff/NGII_1970_EPSG_4326.tif"
outdr = "./drive/MyDrive/Maps/cropped/"

### 1.2.1. Clip and Save as PNG format<br>
---
If you run this in colab, it is safe to mount the Drive before running the code below as the session might get disconnected before 1.2.2.

In [ ]:
# Clip maps by 2㎢ grid

pop_grid=fiona.open(pop_grid_dr)

print(len(pop_grid))

for i in notebook.tqdm(range(len(pop_grid))) :

   mask = gpd.GeoDataFrame.from_features([pop_grid[i]]).set_crs(CRS)

   column=mask["id"]
   number=column[0]
   
   mask.to_file(mask_dr + "{}.shp".format(number.astype(int)))

   mold = mask_dr + '%s.shp' % i
   output= outdr + '%s.png' % i  ## Directly Saving in .png format
   layer=str(i)

   try : 
        !gdalwarp -overwrite -of GTiff -cutline {mold} -cl {layer} -crop_to_cutline -co COMPRESS=DEFLATE -co PREDICTOR=2 -co ZLEVEL=9 {raster} {output}
        
   except: 
     continue
     print(i)


### 1.2.2. Convert Black Pixels to Transparent 

In [ ]:
import os, glob
import cv2
import re
import numpy as np
import pandas as pd
from tqdm import notebook

In [ ]:
# Collect All Map Images in the Drive

png_dr="./drive/MyDrive/Maps/cropped/"
result_dr='./drive/MyDrive/Maps/cropped_black_to_transparent/'

all_pngs=glob.glob(png_dr+ "*.png")

In [38]:
for png in notebook.tqdm(remaining_pngs): ## due to session disconnect, I rerun the same code for files that are not coverted yet (original list: all_pngs)

  gid=re.split("\/|\.", png)[-2]
  
  img=cv2.imread(png)
  color=(0,0,0)
  
  try:
    mask = np.where((img==color).all(axis=2), 0, 255).astype(np.uint8)
    result = img.copy()
    result = cv2.cvtColor(result, cv2.COLOR_BGR2BGRA)
    result[:, :, 3] = mask
    cv2.imwrite(result_dr + "{}.png".format(gid), result)
  
  except: 
    continue
    print(gid)

  0%|          | 0/9791 [00:00<?, ?it/s]